In [1]:
from functools import partial
from typing import List, Tuple, Union

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import HTML, display, update_display

In [2]:
X = +1
O = -1
EMPTY_SPACE = 0

INTERFACE_MAPPING = {
    X: 'X',
    O: 'O'
}

In [3]:
class Interface:
    def __init__(self):
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='100px', height='100px')
                )
                for _ in range(3)
            ]
            for _ in range(3)
        ]

        self.board_widget = VBox([HBox(row) for row in self.buttons])

        self.__player = X
        self.link_positions()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if not button.description:
            button.description = INTERFACE_MAPPING.get(self.__player, '')
            self.__player *= -1  # invert player

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True


    def start(self):
        """Display board interface"""
        display(self.board_widget)

In [4]:
interface = Interface()
interface.start()

In [5]:
def get_game_status(board: np.ndarray) -> Tuple[bool, Union[int, None]]:
    """Check if game ended and who is the winner

    :param board:
    :return: if its a game over, who is the winner
    """
    # check if there is 3 of the same symbol in ...
    for three_player in {3 * X, 3 * O}:
        if ((board.sum(axis=0) == three_player).any() or  # any row
            (board.sum(axis=1) == three_player).any() or  # any col
            (np.diagonal(board).sum() == three_player) or  # main diag
            (np.diagonal(np.fliplr(board)).sum() == three_player)):  # sec diag
            return True, three_player // 3

    return (board == EMPTY_SPACE).sum() == 0, None

In [6]:
def get_possible_moves(board: np.ndarray, player: int = X) -> List[np.ndarray]:
    """Get next possible moves by some player
    """
    def get_board_after_move(position):
        new_board = board.copy()
        new_board[position[0], position[1]] = player
        return new_board

    empty_positions = np.argwhere(board == EMPTY_SPACE)
    return [get_board_after_move(position) for position in empty_positions]

In [7]:
visited_boards = {}


def convert_to_key(board: np.ndarray, player:int) -> tuple:
    """Turn board hashable"""
    return tuple(map(tuple, board)), player


def dynamic_program(function):
    """Avoids duplicated calculations"""
    def wrapper(board, player, *args):
        key = convert_to_key(board, player)

        if key in visited_boards:
            return visited_boards[key]

        result = function(board, player, *args)
        visited_boards[key] = result

        return result

    return wrapper

In [8]:
AI_PLAYER = X
HUMAN_PLAYER = O
MAX_N_MOVES = 9


def get_score(winner: int, n_moves: int) -> int:
    """Get how well was the game for the AI:
        - win faster is better than win slower
        - lose slower is better than win faster
        - draw is a intermediary result
    """
    score = MAX_N_MOVES + 1 - n_moves
    return score if winner == AI_PLAYER else (-score if winner else 0)


In [9]:
@dynamic_program
def mini_max(board, player=AI_PLAYER, n_moves=0):
    is_max_step = player == AI_PLAYER

    is_end_game, winner = get_game_status(board)
    if is_end_game:
        return get_score(winner, n_moves), board

    best_score, best_move = None, None
    for new_board in get_possible_moves(board, player):
        score, _ = mini_max(new_board, -player, n_moves+1)

        if (best_score is None or  # first iteration
            is_max_step and score > best_score or  # is maximization step and score > previous best
            not is_max_step and score < best_score):  # is minization step and score < previous best
            best_score, best_move = score, new_board

    return best_score, best_move

In [10]:
board = np.zeros(shape=(3, 3), dtype=np.int8)
_, new_board = mini_max(board, AI_PLAYER)
new_board

array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [11]:
new_board = new_board.copy()
new_board[1, 0] = -1
_, new_board = mini_max(new_board, AI_PLAYER)
new_board

array([[ 1,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  0]], dtype=int8)

In [14]:
END_GAME_MESSAGES = {
    AI_PLAYER: 'AI won!',
    HUMAN_PLAYER: 'You won!',
    EMPTY_SPACE: "It is a draw"
}


class TicTacToeAI(Interface):
    def __init__(self):
        self.board = np.zeros(shape=(3, 3), dtype=np.int8)
        super().__init__()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board
        It calls minmax algorithm after each user move

        :param pos: row, column of button on grid
        :param button: clicked button"""
        if not button.description:
            self.board[pos] = HUMAN_PLAYER
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()

            is_over, winner = get_game_status(self.board)
            if is_over:
                self.disable_buttons()
                print(END_GAME_MESSAGES.get(winner))

            self.update()

    def update(self):
        """Update interface from virtual board"""
        for i, row in enumerate(self.board):
            for j, item in enumerate(row):
                self.buttons[i][j].description = INTERFACE_MAPPING.get(item, '')


In [15]:
game = TicTacToeAI()
game.start()

AI won!
